In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost


Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix_car


In [0]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

Feature Engineering

In [0]:
SUFFIX_CAT = '_cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values


  
  scores = cross_val_score(model,X, y, cv=3, scoring='neg_mean_absolute_error' )
  return np.mean(scores), np.std(scores)

DecisionTree

In [0]:
run_model ( DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

## Random Forest

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

## XGBoost

In [0]:
xgb_params = {
    'max_depth' : 5,
    'n_estimators' :50,
    'leraning_rate':0.1,
    'seed': 0
}
run_model(xgb.XGBRFRegressor(**xgb_params), cat_feats)

[06:12:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:12:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:13:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-18133.227710015504, 8.30184918497527)

In [0]:
m = xgb.XGBRFRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(X,y)


imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[06:15:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.0445 ± 0.0005,param_napęd_cat
0.0284 ± 0.0009,param_rok-produkcji_cat
0.0269 ± 0.0005,param_skrzynia-biegów_cat
0.0258 ± 0.0007,param_faktura-vat_cat
0.0135 ± 0.0004,param_moc_cat
0.0073 ± 0.0003,param_stan_cat
0.0064 ± 0.0002,feature_kamera-cofania_cat
0.0041 ± 0.0003,param_pojemność-skokowa_cat
0.0032 ± 0.0000,feature_łopatki-zmiany-biegów_cat
0.0016 ± 0.0000,feature_światła-led_cat


In [0]:
len(cat_feats)

151

In [0]:
 	feats = ['param_napęd_cat',
 	'param_rok-produkcji_cat',
 	'param_skrzynia-biegów_cat',
 	'param_faktura-vat_cat',
 	'param_moc_cat',
 	'param_stan_cat',
 	'feature_kamera-cofania_cat',
 	'param_pojemność-skokowa_cat',
 	'feature_łopatki-zmiany-biegów_cat',
 	'feature_światła-led_cat',
 	'feature_regulowane-zawieszenie_cat',
 	'seller_type_cat',
 	'feature_wspomaganie-kierownicy_cat',
 	'seller_name_cat',
 	'feature_elektrycznie-ustawiane-fotele_cat',
 	'param_wersja_cat',
 	'feature_system-start-stop_cat',
 	'feature_bluetooth_cat',
 	'param_typ_cat',
 	'feature_czujniki-parkowania-przednie_cat']

run_model(xgb.XGBRFRegressor(**xgb_params),feats)

[07:27:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:27:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:27:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-18245.066872632644, 49.11489313212736)

In [0]:
run_model(xgb.XGBRFRegressor(**xgb_params),feats)

[07:15:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:15:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:15:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-18245.066872632644, 49.11489313212736)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

feats = ['param_napęd_cat',
 	'param_rok-produkcji',
 	'param_skrzynia-biegów_cat',
 	'param_faktura-vat_cat',
 	'param_moc_cat',
 	'param_stan_cat',
 	'feature_kamera-cofania_cat',
 	'param_pojemność-skokowa_cat',
 	'feature_łopatki-zmiany-biegów_cat',
 	'feature_światła-led_cat',
 	'feature_regulowane-zawieszenie_cat',
 	'seller_type_cat',
 	'feature_wspomaganie-kierownicy_cat',
 	'seller_name_cat',
 	'feature_elektrycznie-ustawiane-fotele_cat',
 	'param_wersja_cat',
 	'feature_system-start-stop_cat',
 	'feature_bluetooth_cat',
 	'param_typ_cat',
 	'feature_czujniki-parkowania-przednie_cat']


run_model(xgb.XGBRFRegressor(**xgb_params),feats)

[07:19:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:19:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:19:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-14812.118929065253, 113.41383245551182)

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int (str (x).split(' ')[0]))

df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

feats = ['param_napęd_cat',
 	'param_rok-produkcji',
 	'param_skrzynia-biegów_cat',
 	'param_faktura-vat_cat',
 	'param_moc',
 	'param_stan_cat',
 	'feature_kamera-cofania_cat',
 	'param_pojemność-skokowa_cat',
 	'feature_łopatki-zmiany-biegów_cat',
 	'feature_światła-led_cat',
 	'feature_regulowane-zawieszenie_cat',
 	'seller_type_cat',
 	'feature_wspomaganie-kierownicy_cat',
 	'seller_name_cat',
 	'feature_elektrycznie-ustawiane-fotele_cat',
 	'param_wersja_cat',
 	'feature_system-start-stop_cat',
 	'feature_bluetooth_cat',
 	'param_typ_cat',
 	'feature_czujniki-parkowania-przednie_cat']


run_model(xgb.XGBRFRegressor(**xgb_params),feats)

[07:23:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:23:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:23:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13215.121413649184, 42.92904171535839)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int( str (x).split('cm')[0].replace(' ', '')))

feats = ['param_napęd_cat',
 	'param_rok-produkcji',
 	'param_skrzynia-biegów_cat',
 	'param_faktura-vat_cat',
 	'param_moc',
 	'param_stan_cat',
 	'feature_kamera-cofania_cat',
 	'param_pojemność-skokowa',
 	'feature_łopatki-zmiany-biegów_cat',
 	'feature_światła-led_cat',
 	'feature_regulowane-zawieszenie_cat',
 	'seller_type_cat',
 	'feature_wspomaganie-kierownicy_cat',
 	'seller_name_cat',
 	'feature_elektrycznie-ustawiane-fotele_cat',
 	'param_wersja_cat',
 	'feature_system-start-stop_cat',
 	'feature_bluetooth_cat',
 	'param_typ_cat',
 	'feature_czujniki-parkowania-przednie_cat']


run_model(xgb.XGBRFRegressor(**xgb_params),feats)

[07:23:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:23:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:23:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13010.490321206844, 54.98747354225763)